In [1]:
from tqdm import tqdm

In [41]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
from pyltp import Segmentor

In [4]:
import os

In [5]:
import re

In [30]:
from gensim.models import KeyedVectors

In [193]:
from tensorflow.python.keras.models import Sequential

In [196]:
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional

In [209]:
from tensorflow.python.keras.optimizers import Adam

In [214]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [7]:
fname = '..\\Desktop\\movie_comments.csv'

In [8]:
comments = pd.read_csv(fname,encoding='utf8')

C:\Users\672\.conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
comments

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1
5,6,https://movie.douban.com/subject/26363254/,战狼2,“犯我中华者，虽远必诛”，吴京比这句话还要意淫一百倍。,1
6,7,https://movie.douban.com/subject/26363254/,战狼2,脑子是个好东西，希望编剧们都能有。,2
7,8,https://movie.douban.com/subject/26363254/,战狼2,三星半，实打实的7分。第一集在爱国主旋律内部做着各种置换与较劲，但第二集才真正显露吴京的野心...,4
8,9,https://movie.douban.com/subject/26363254/,战狼2,开篇长镜头惊险大气引人入胜 结合了水平不俗的快剪下实打实的真刀真枪 让人不禁热血沸腾 特别弹...,4
9,10,https://movie.douban.com/subject/26363254/,战狼2,15/100吴京的冷峰在这部里即像成龙，又像杰森斯坦森，但体制外的同类型电影，主角总是代表个...,1


In [25]:
len(comments)

261497

In [26]:
Survived_freq = comments.star.value_counts()
print(Survived_freq)

4       43002
4       40806
3       33910
5       31947
3       31764
5       27368
2       14299
2       13837
1       12308
1       12255
star        1
Name: star, dtype: int64


In [63]:
a = comments.star[9]
type(a)

str

In [64]:
b = comments.star[10]
type(b)

str

## 分词

In [11]:
content_X = comments.comment.dropna().values.tolist()

In [49]:
content_X[:5]

['吴京意淫到了脑残的地步，看了恶心想吐',
 '首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮番上场，视物理逻辑于不顾，不得不说有钱真好，随意胡闹',
 '吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋律，为了煽情而煽情，让人觉得他是个大做作、大谎言家。（7.29更新）片子整体不如湄公河行动，1.整体不够流畅，编剧有毒，台词尴尬；2.刻意做作的主旋律煽情显得如此不合时宜而又多余。',
 '凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。',
 '中二得很']

In [12]:
model_path = 'C:\\Users\\672\\Desktop\\ltp_data_v3.4.0'
cws_model_path = os.path.join(model_path, 'cws.model')

In [13]:
segmentor = Segmentor()
segmentor.load(cws_model_path)

In [41]:
# segment=[]
# for line in tqdm(content_X):
#     try:
#         segs = segmentor.segment(line) # jiaba.lcut()
#         for seg in segs:
#             if len(seg)>1 and seg!='\r\n':
#                  segment.append(seg)
#     except:
#         print(line)
#         continue

100%|████████████████████████████████████████████████████████████████████████| 261495/261495 [01:55<00:00, 2270.90it/s]


In [15]:
table_cut = []
mask = []
for sentence in tqdm(content_X):
    if not isinstance(sentence, str):
        mask.append(False)
        continue
    mask.append(True)
    sentence = ''.join(re.findall(r'\w+', sentence))
    table_cut.append(' '.join(segmentor.segment(sentence)))
len(table_cut)

100%|████████████████████████████████████████████████████████████████████████| 261495/261495 [01:43<00:00, 2525.91it/s]


261495

In [55]:
table_cut[:5]

['吴京意 淫 到 了 脑残 的 地步 看 了 恶心 想吐',
 '首映礼 看 的 太 恐怖 了 这个 电影 不 讲 道理 的 完全 就是 吴 京 在 实现 他 这个 小 粉红 的 英雄 梦 各种 装备 轮番 上场 视 物理 逻辑 于 不顾 不得不 说 有 钱 真 好 随意 胡闹',
 '吴 京 的 炒作 水平 不 输 冯 小刚 但 小刚 至少 不 会 用 主旋律 来 炒作 吴 京 让 人 看 了 不 舒服 为了 主旋律 而 主旋律 为了 煽情 而 煽情 让 人 觉得 他 是 个 大 做 作 大谎言 家 729 更新 片子 整体 不如 湄公河 行动 1 整体 不 够 流畅 编剧 有毒 台词 尴尬 2 刻意 做作 的 主旋律 煽情 显得 如此 不合时宜 而 又 多余',
 '凭 良心 说 好 看到 不 像 战狼1 的 续集 完虐 湄公河 行动',
 '中 二 得 很']

## 去停用词

In [16]:
table_cut_0 = pd.DataFrame({'segment':table_cut})

In [61]:
table_cut_0.head()

,segment
0,吴京意 淫 到 了 脑残 的 地步 看 了 恶心 想吐
1,首映礼 看 的 太 恐怖 了 这个 电影 不 讲 道理 的 完全 就是 吴 京 在 实现 他...
2,吴 京 的 炒作 水平 不 输 冯 小刚 但 小刚 至少 不 会 用 主旋律 来 炒作 吴 ...
3,凭 良心 说 好 看到 不 像 战狼1 的 续集 完虐 湄公河 行动
4,中 二 得 很


In [18]:
 stopwords = pd.read_csv("./stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'],encoding='utf-8') 

In [19]:
table_cut_1=table_cut_0[~table_cut_0.segment.isin(stopwords.stopword)]

In [20]:
len(table_cut_0),len(table_cut_1)

(261495, 260173)

In [38]:
def getWordVecs(wordList):
    vecs = []
    for word in wordList:
        try:
            vecs.append(cn_model[word])
        except KeyError:
            continue
    # vecs = np.concatenate(vecs)
    return np.array(vecs, dtype = 'float')

In [35]:
def buildVecs(filename):
    posInput = []
    with open(filename, "rb") as txtfile:
        # print txtfile
        for lines in txtfile:
            lines = lines.split('\n ')
            for line in lines:            
                line = jieba.cut(line)
                resultList = getWordVecs(line)
                # for each sentence, the mean vector of all its vectors is used to represent this sentence
                if len(resultList) != 0:
                    resultArray = sum(np.array(resultList))/len(resultList)
                    posInput.append(resultArray)

In [33]:
cn_model = KeyedVectors.load_word2vec_format('./chinese_sentiment-master/sgns.zhihu.bigram', binary=False)

C:\Users\672\.conda\envs\tensorflow\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [157]:
posInput = []
for line in tqdm(table_cut_1.segment[:50]):            
    resultList = getWordVecs(line)
    if len(resultList) != 0:
        resultArray = sum(np.array(resultList))/len(resultList)
        posInput.append(resultArray)

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 781.70it/s]


In [57]:
posInput[1]

array([ 0.03323672,  0.07659372, -0.22908188,  0.15992821, -0.15013271,
       -0.28016315, -0.23964886,  0.0105634 ,  0.05356391, -0.14371899,
       -0.24426919,  0.34718954,  0.05880805,  0.049607  ,  0.24275369,
       -0.55603327,  0.07609777,  0.05125983,  0.17479677, -0.08144276,
        0.20585264,  0.14045714,  0.01676063, -0.41771758, -0.20670643,
        0.37567842, -0.09244092, -0.32330343,  0.15953442,  0.27752497,
       -0.10727489, -0.00221989,  0.0730382 ,  0.13129021,  0.16399334,
        0.01446586,  0.28158207,  0.41305294, -0.04603875,  0.10524286,
       -0.25630337, -0.04626588, -0.185317  ,  0.22887774,  0.18249705,
        0.12801188, -0.12251343,  0.12248529,  0.10352026,  0.04089618,
       -0.08989192, -0.18186422, -0.14976605,  0.21961005, -0.21362955,
       -0.19295132, -0.20339512,  0.31055332,  0.14497954, -0.17854002,
        0.08379952,  0.37250842, -0.24160069, -0.18440032,  0.10504488,
        0.03636578,  0.35068785,  0.27440283,  0.09784431,  0.15

In [158]:
posiInput = posInput
# negeInput = posInput

In [49]:
# y = np.concatenate((np.ones(len(posiInput)), np.zeros(len(negeInput))))

In [54]:
# y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [159]:
X = posInput[:]
for neg in negInput:
    X.append(neg)
X = np.array(X)

In [160]:
X

array([[-0.02121247,  0.30451006, -0.17747165, ..., -0.00193494,
        -0.15720882, -0.350485  ],
       [ 0.03323672,  0.07659372, -0.22908188, ...,  0.0548705 ,
         0.09271856, -0.27019792],
       [ 0.06019704,  0.10150806, -0.17916154, ..., -0.00169732,
         0.11349135, -0.20233715],
       ...,
       [ 0.05955349, -0.13202932,  0.06808319, ..., -0.02109149,
         0.14557695, -0.17230568],
       [ 0.11066596,  0.03213573, -0.21363617, ...,  0.02164995,
         0.03919675, -0.12997216],
       [ 0.20960426,  0.124947  , -0.12515197, ..., -0.0185439 ,
        -0.00282341, -0.36609331]])

In [161]:
X.shape

(100, 300)

In [162]:
y = comments['star'][:100]

In [151]:
len(y)

50

In [145]:
type(y)

pandas.core.series.Series

In [147]:
from sklearn.model_selection import train_test_split

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [189]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((67, 300), (33, 300), (67,), (33,))

In [164]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [186]:
def get_performance(clf, x_, y_):
    y_hat = clf.predict(x_)
    #print('f1_score is: {}'.format(f1_score(y_, y_hat)))
    print('accuracy is: {}'.format(accuracy_score(y_, y_hat)))
    #print('percision is: {}'.format(precision_score(y_, y_hat)))
    #print('recall is: {}'.format(recall_score(y_, y_hat)))

In [174]:
from sklearn.linear_model import LogisticRegressionCV,LinearRegression,LogisticRegression

## 尝试了一下普通logic

In [187]:
def Logistic(X_train, y_train):
    clf = LogisticRegression(fit_intercept=True, penalty='l2', solver='lbfgs', tol=0.01, multi_class='multinomial')
    clf.fit(X_train, y_train)
    return clf
#clf.fit(X_test,y_test)
get_performance(Logistic(X_train,y_train),X_test,y_test)

accuracy is: 0.15151515151515152


## 试试看Neural Network

In [194]:
model = Sequential()

In [281]:
num_words = 50
embedding_dim = 300
embedding_dim

300

In [282]:
embedding_matrix = np.zeros((num_words, embedding_dim))
# embedding_matrix为一个 [num_words，embedding_dim] 的矩阵
# 维度为 50 * 300
for i in range(num_words):
    embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]
embedding_matrix = embedding_matrix.astype('float32')
max_tokens=300

In [ ]:
# 模型第一层为embedding
model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))

In [248]:
model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
model.add(LSTM(units=16, return_sequences=False))

In [249]:
model.add(Dense(1, activation='sigmoid'))
# 我们使用adam以0.001的learning rate进行优化
optimizer = Adam(lr=1e-3)

In [250]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [258]:
# 我们来看一下模型的结构，一共90k左右可训练的变量
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 300)           15000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 64)            85248     
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                5184      
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1, 50)             2500000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 64)             21248     
__________

In [252]:
# 建立一个权重的存储点
path_checkpoint = 'sentiment_checkpoint.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [253]:
# 尝试加载已训练模型
try:
    model.load_weights(path_checkpoint)
except Exception as e:
    print(e)

Unable to open file (unable to open file: name = 'sentiment_checkpoint.keras', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [254]:
# 定义early stoping如果3个epoch内validation loss没有改善则停止训练
earlystopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [255]:
# 自动降低learning rate
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-5, patience=0,
                                       verbose=1)

In [256]:
# 定义callback函数
callbacks = [
    earlystopping, 
    checkpoint,
    lr_reduction
]

In [43]:
# #words_df = pd.DataFrame({'segment':segment})
# stopwords = pd.read_csv("./stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'],encoding='utf-8') 

In [66]:
# stopwords.head()

,stopword
0,!
1,""""
2,#
3,$
4,%


In [62]:
# table_cut_1=table_cut_0[~table_cut_0.segment.isin(stopwords.stopword)]

In [67]:
# table_cut_1.head()

,segment
0,吴京意 淫 到 了 脑残 的 地步 看 了 恶心 想吐
1,首映礼 看 的 太 恐怖 了 这个 电影 不 讲 道理 的 完全 就是 吴 京 在 实现 他...
2,吴 京 的 炒作 水平 不 输 冯 小刚 但 小刚 至少 不 会 用 主旋律 来 炒作 吴 ...
3,凭 良心 说 好 看到 不 像 战狼1 的 续集 完虐 湄公河 行动
4,中 二 得 很


In [56]:
# words_df[:5]

,segment
0,吴京意
1,脑残
2,地步
3,恶心
4,想吐
